In [131]:
import pandas as pd
import numpy as np
import plotly.express as px

In [132]:
path = '../data/raw/EventSalesReportDetailed-2024-02-29T12_29_51.csv'

In [133]:
df = pd.read_csv(path)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12975 entries, 0 to 12974
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Event Name          12975 non-null  object 
 1   Event Date/Time     12975 non-null  object 
 2   Is Comp             10158 non-null  float64
 3   Seat Status         12975 non-null  object 
 4   Ticket Type         10158 non-null  object 
 5   Ticket Band         10158 non-null  object 
 6   Lock Type           1541 non-null   object 
 7   Sales Channel       10158 non-null  object 
 8   User Confirmed      10158 non-null  object 
 9   Price Overridden    10158 non-null  object 
 10  Area                12975 non-null  object 
 11  Sum of Price        10158 non-null  float64
 12  Sum of Gross Value  12975 non-null  float64
 13  Count               12975 non-null  int64  
dtypes: float64(3), int64(1), object(10)
memory usage: 1.4+ MB


,Event Name,Event Date/Time,Is Comp,Seat Status,Ticket Type,Ticket Band,Lock Type,Sales Channel,User Confirmed,Price Overridden,Area,Sum of Price,Sum of Gross Value,Count
0,1012 Raghu Dixit,10/07/2023 19:30,NaN,Available,NaN,NaN,NaN,NaN,NaN,NaN,Main Hall,NaN,4360.0,218
1,1012 Raghu Dixit,10/07/2023 19:30,NaN,Locked,NaN,NaN,General,NaN,NaN,NaN,Main Hall,NaN,5000.0,250
2,1012 Raghu Dixit,10/07/2023 19:30,0.0,Sold,Full Price,Price band A,NaN,Counter,chloe.thomas,False,Main Hall,160.0,160.0,8
3,1012 Raghu Dixit,10/07/2023 19:30,0.0,Sold,Full Price,Price band A,NaN,Counter,katina.kilkelly,False,Main Hall,160.0,160.0,8
4,1012 Raghu Dixit,10/07/2023 19:30,0.0,Sold,Full Price,Price band A,NaN,Counter,Luke,False,Main Hall,120.0,120.0,6


In [134]:
df['Event Date/Time'] = pd.to_datetime(df['Event Date/Time'], format = '%d/%m/%Y %H:%M')
df['Event Date'] = df['Event Date/Time'].dt.date
df['Year'] = df['Event Date/Time'].dt.year
df['Month'] = df['Event Date/Time'].dt.month

In [136]:
import pandas as pd
from datetime import datetime

# Get the current time
current_time = datetime.now()

# Filter the DataFrame to keep only rows with datetime values greater than or equal to the current time
df_filtered = df[df['Event Date/Time'] >= current_time]



In [143]:
df_filtered[df_filtered['Event Name'] == 'Sack Sabbath'].values_count()

,Event Name,Event Date/Time,Is Comp,Seat Status,Ticket Type,Ticket Band,Lock Type,Sales Channel,User Confirmed,Price Overridden,Area,Sum of Price,Sum of Gross Value,Count,Event Date,Year,Month
9538,Sack Sabbath,2024-04-26 19:00:00,NaN,Available,NaN,NaN,NaN,NaN,NaN,NaN,The Tree House,NaN,1302.0,84,2024-04-26,2024,4
9539,Sack Sabbath,2024-04-26 19:00:00,NaN,Locked,NaN,NaN,General,NaN,NaN,NaN,The Tree House,NaN,620.0,40,2024-04-26,2024,4
9540,Sack Sabbath,2024-04-26 19:00:00,0.0,Sold,Full Price,Price band A,NaN,Counter,olivia.dimery,False,The Tree House,15.5,15.5,1,2024-04-26,2024,4
9541,Sack Sabbath,2024-04-26 19:00:00,0.0,Sold,Full Price,Price band A,NaN,Web,web,False,The Tree House,387.5,387.5,25,2024-04-26,2024,4


In [156]:
sales_channel_counts = df['Sales Channel'].value_counts()
print(sales_channel_counts.values)

[6240 2164 1754]


In [73]:
from datetime import datetime

df_future = df[(df['Event Date/Time'])>=datetime.now()]
df_max = df_future[df_future['Seat Status'] !='Locked']
df_current = df_future[df_future['Seat Status'] =='Sold']

df_current = df_current.groupby(['Event Date/Time', 'Event Name']).agg({'Count': 'sum'}).reset_index()
df_max = df_max.groupby(['Event Date/Time', 'Event Name']).agg({'Count': 'sum'}).reset_index()

In [108]:
df_sales = pd.merge(df_max,df_current, on = ['Event Date/Time', 'Event Name'], suffixes = ('_max', '_current'))
df_sales['Count_avail'] = df_sales['Count_max'] - df_sales['Count_current']
df_sales['Percent_sold'] = df_sales['Count_current'] / df_sales['Count_max']
df_sales['Percent_avail'] = df_sales['Count_avail'] / df_sales['Count_max']
df_sales['Event Name-Date'] = df_sales['Event Name'] + df_sales['Event Date/Time'].dt.strftime(' (%Y-%m-%d %H:%M)')
df_sales[df_sales['Event Name'] == 'Amazing Animals']

,Event Date/Time,Event Name,Count_max,Count_current,Count_avail,Percent_sold,Percent_avail,Event Name-Date
9,2024-05-06 10:30:00,Amazing Animals,239,129,110,0.539749,0.460251,Amazing Animals (2024-05-06 10:30)
10,2024-05-06 13:30:00,Amazing Animals,355,346,9,0.974648,0.025352,Amazing Animals (2024-05-06 13:30)


In [109]:
import plotly.express as px

# Create a stacked bar chart
fig = px.bar(df_sales, x='Event Name-Date', y=['Count_current', 'Count_avail'], color_discrete_sequence=['#636EFA', '#EF553B'],
             title='Current and Available Counts by Event', barmode='relative')

# Set the x-axis labels to be rotated for better readability
fig.update_layout(xaxis_tickangle=-45)

# Show the chart
fig.show()


In [110]:
# Create a stacked bar chart
fig = px.bar(df_sales, x='Event Name-Date', y=['Percent_sold', 'Percent_avail'], color_discrete_sequence=['#636EFA', '#EF553B'],
             title='Current and Available Counts by Event', barmode='relative')

# Set the x-axis labels to be rotated for better readability
fig.update_layout(xaxis_tickangle=-45)

# Show the chart
fig.show()

In [116]:
df_sales['Event Name-Date'].values

array(['Sack Sabbath (2024-04-26 19:00)', 'Paul Young (2024-04-27 19:30)',
       'Jake Bugg (2024-05-02 19:00)',
       'Ian Botham & Geoff Miller (2024-05-03 19:00)',
       'Pet Needs (2024-05-04 19:00)',
       'The Cuban Brothers / DJ Yoda (2024-05-04 19:00)',
       'Magpie Market 2024 (2024-05-05 09:00)',
       'Clinton Baptiste : Roller Ghoster! (2024-05-05 19:00)',
       'Ríoghnach Connolly & Honeyfeet (2024-05-05 19:30)',
       'Amazing Animals (2024-05-06 10:30)',
       'Amazing Animals (2024-05-06 13:30)',
       'The Orielles (2024-05-07 19:30)',
       'New Purple Celebration - The Music of Prince (2024-05-10 19:00)',
       'Gary Stringer (2024-05-10 19:30)',
       'Gary Stringer (2024-05-11 19:30)',
       'Kings of Lyon & The UK Strokes (2024-05-11 19:30)',
       'Pale Blue Eyes (2024-05-14 19:00)',
       'John Lydon (2024-05-18 19:00)', 'ZZ TOPPD (2024-05-18 19:30)',
       'Jeremy Healy (2024-05-25 19:00)', 'Zion Train (2024-05-31 19:00)',
       'Charlie Simp

In [113]:
yy = df_sales['Event Name-Date'].unique()
labels = [q.split(' (')[0] for q in df_sales['Event Name-Date'].unique()]
labels

['Sack Sabbath',
 'Paul Young',
 'Jake Bugg',
 'Ian Botham & Geoff Miller',
 'Pet Needs',
 'The Cuban Brothers / DJ Yoda',
 'Magpie Market 2024',
 'Clinton Baptiste : Roller Ghoster!',
 'Ríoghnach Connolly & Honeyfeet',
 'Amazing Animals',
 'Amazing Animals',
 'The Orielles',
 'New Purple Celebration - The Music of Prince',
 'Gary Stringer',
 'Gary Stringer',
 'Kings of Lyon & The UK Strokes',
 'Pale Blue Eyes',
 'John Lydon',
 'ZZ TOPPD',
 'Jeremy Healy',
 'Zion Train',
 'Charlie Simpson DJ Set',
 'Magpie Market 2024',
 'Buzzcocks',
 'Toyah & Robert',
 'The Blockheads 2024',
 'Troy Redfern',
 'Oye Santana',
 'London Calling',
 'Magpie Market 2024',
 'Preacher Stone',
 'Wrecking Ball',
 'Magpie Market 2024',
 'Ion Maiden',
 'Magpie Market 2024',
 'The Devout',
 'The South',
 'Chantel McGregor',
 'The Entitled Sons',
 'R.E.M by Stipe',
 'Kerry Ellis - Queen of the West End',
 "Jo Whiley's 90s Anthems",
 'Brave Rival',
 'Magpie Market 2024',
 'Steve Ignorant Band',
 'LINDISFARNE',
 'Walt

In [117]:
df_sales[df_sales['Event Name'] == 'Sack Sabbath']

,Event Date/Time,Event Name,Count_max,Count_current,Count_avail,Percent_sold,Percent_avail,Event Name-Date
0,2024-04-26 19:00:00,Sack Sabbath,110,26,84,0.236364,0.763636,Sack Sabbath (2024-04-26 19:00)


In [127]:
np.append(df_sales[df_sales['Event Name'] == 'Sack Sabbath']['Count_current'].values, df_sales[df_sales['Event Name'] == 'Sack Sabbath']['Count_avail'].values)

array([26, 84], dtype=int64)

In [129]:
import plotly.graph_objects as go

fig3 = go.Figure(data = [go.Pie(labels = ['Count_current', 'Count_avail'], values = np.append(df_sales[df_sales['Event Name'] == 'Sack Sabbath']['Count_current'].values, df_sales[df_sales['Event Name'] == 'Sack Sabbath']['Count_avail'].values), title = 'Current and Available Counts for Sack Sabbath')])
fig3.show()

In [38]:

def create_price_line_graph(df):
    # Group the dataframe by 'Month' and 'Year' and sum up the 'Sum of Price' column
    df_sum = df.groupby(['Month', 'Year'])['Sum of Price'].sum().reset_index()

    # Create a line graph using plotly.express
    fig = px.line(df_sum, x='Month', y='Sum of Price', color='Year')

    # Show the graph
    fig.show()

In [41]:
df_sum = df.groupby(['Year', 'Month'])['Sum of Price'].sum().reset_index()
df_sum['Year-Month'] = df_sum['Year'].astype(str) + '-' + df_sum['Month'].astype(str)
df_sum.head()
#     # Create a line graph using plotly.express
# fig = px.line(df_sum, x='Month', y='Sum of Price', color='Year')

#     # Show the graph
# fig.show()

,Year,Month,Sum of Price,Year-Month
0,2015,1,23910.50,2015-1
1,2015,2,26996.22,2015-2
2,2015,3,26073.50,2015-3
3,2015,4,37244.00,2015-4
4,2015,5,37439.04,2015-5


Line graph (basic overall) 
Line (might wanna do colors in the future) 


In [130]:
df_filtered

NameError: name 'df_filtered' is not defined

In [173]:
import os

df = pd.read_csv('./raw/EventSalesReportDetailed-2024-04-26T13_45_32.csv')

In [174]:
df['Event Date/Time'] = pd.to_datetime(df['Event Date/Time'], format = '%d/%m/%Y %H:%M')
df['Event Date'] = df['Event Date/Time'].dt.date
df['Year'] = df['Event Date/Time'].dt.year
df['Month'] = df['Event Date/Time'].dt.month
current_time = datetime.now()
df_filtered = df[df['Event Date/Time'] >= current_time]

In [176]:
df

,Event Name,Event Date/Time,Is Comp,Seat Status,Ticket Type,Ticket Band,Lock Type,Sales Channel,User Confirmed,Price Overridden,Area,Sum of Price,Sum of Gross Value,Count,Event Date,Year,Month
0,A Band Called Malice,2024-04-06 19:00:00,NaN,Locked,NaN,NaN,General,NaN,NaN,NaN,The Tree House,NaN,660.0,40,2024-04-06,2024,4
1,A Band Called Malice,2024-04-06 19:00:00,0.0,Sold,Full Price,Price band A,NaN,Counter,georgie.rose,False,The Tree House,49.5,49.5,3,2024-04-06,2024,4
2,A Band Called Malice,2024-04-06 19:00:00,0.0,Sold,Full Price,Price band A,NaN,Counter,jayne.chappell,False,The Tree House,66.0,66.0,4,2024-04-06,2024,4
3,A Band Called Malice,2024-04-06 19:00:00,0.0,Sold,Full Price,Price band A,NaN,Counter,katina.kilkelly,False,The Tree House,49.5,49.5,3,2024-04-06,2024,4
4,A Band Called Malice,2024-04-06 19:00:00,0.0,Sold,Full Price,Price band A,NaN,Counter,Lou.Payne,False,The Tree House,33.0,33.0,2,2024-04-06,2024,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,Wham!Duran,2024-04-12 19:30:00,0.0,Sold,Full Price,Price band A,NaN,Counter,olivia.dimery,False,Cheese & Grain Standing,210.0,210.0,12,2024-04-12,2024,4
767,Wham!Duran,2024-04-12 19:30:00,0.0,Sold,Full Price,Price band A,NaN,Phone,katina.kilkelly,False,Cheese & Grain Standing,105.0,105.0,6,2024-04-12,2024,4
768,Wham!Duran,2024-04-12 19:30:00,0.0,Sold,Full Price,Price band A,NaN,Web,web,False,Cheese & Grain Standing,5022.5,5022.5,287,2024-04-12,2024,4
769,Wham!Duran,2024-04-12 19:30:00,1.0,Sold,Comp,Price band A,NaN,Counter,katina.kilkelly,False,Cheese & Grain Standing,0.0,17.5,1,2024-04-12,2024,4


In [172]:
df_future = df[(df['Event Date/Time'])>=datetime.now()]
df_max = df_future[df_future['Seat Status'] !='Locked']
df_current = df_future[df_future['Seat Status'] =='Sold']

df_current = df_current.groupby(['Event Date/Time', 'Event Name']).agg({'Count': 'sum'}).reset_index()
df_max = df_max.groupby(['Event Date/Time', 'Event Name']).agg({'Count': 'sum'}).reset_index()

df_filtered

,Event Name,Event Date/Time,Is Comp,Seat Status,Ticket Type,Ticket Band,Lock Type,Sales Channel,User Confirmed,Price Overridden,Area,Sum of Price,Sum of Gross Value,Count,Event Date,Year,Month


In [169]:

df_sales = pd.merge(df_max,df_current, on = ['Event Date/Time', 'Event Name'], suffixes = ('_max', '_current'))
df_sales['Count_avail'] = df_sales['Count_max'] - df_sales['Count_current']
df_sales['Percent_sold'] = df_sales['Count_current'] / df_sales['Count_max']
df_sales['Percent_avail'] = df_sales['Count_avail'] / df_sales['Count_max']
df_sales['Event Name-Date'] = df_sales['Event Name'] + df_sales['Event Date/Time'].dt.strftime(' (%Y-%m-%d %H:%M)')
df_sales.drop_duplicates(subset = ['Event Name'], inplace = True)

,Count_max,Event Date/Time,Event Name,Count_current,Count_avail,Percent_sold,Percent_avail,Event Name-Date
